### New Working Code

In [10]:
import ee
import pandas as pd
from scipy.spatial import cKDTree

# Initialize Earth Engine
ee.Initialize(project='data690-zhouhaomatt')

# Set display options to show all columns
pd.set_option('display.max_columns', None)

# Define the points of interest (coordinates for which data will be extracted)
points_of_interest = [
    (170.89142642028511, -43.99924808088317),
    (170.89155129827327, -43.99924808088317),
    (170.89167617626143, -43.99924808088317),
    (170.89180105424958, -43.99924808088317),
    (170.89192593223774, -43.99924808088317),
    (170.89192593223774, -43.99915824976567),
    (170.89142642028511, -43.99906841864817),
    (170.89192593223774, -43.99906841864817)
]

# Create EE geometry points (server-side objects)
ee_points = [ee.Geometry.Point(lon, lat) for lon, lat in points_of_interest]

# Define the ROI using a central point and buffer (server-side object)
central_point = ee.Geometry.Point(170.89167617626143, -43.99906841864817)
roi = central_point.buffer(30)  # 30-meter buffer around the central point

# Setup image and cloud score collections (server-side objects)
images = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2017-01-01', '2024-06-25').filterBounds(roi)
clouds = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED').filterDate('2017-01-01', '2024-06-25').filterBounds(roi)

# Data extraction setup
region = ee.Geometry.MultiPoint(ee_points)
scale = 10  # Spatial resolution in meters
band_list = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12']
cloud_bands = ['cs', 'cs_cdf']

# Fetch pixel and cloud values from Earth Engine and bring them to the client side
try:
    pixel_values = images.select(band_list).getRegion(region, scale).getInfo()
    cloud_values = clouds.select(cloud_bands).getRegion(region, scale).getInfo()
except Exception as e:
    print(f"Failed to retrieve data: {e}")
    exit()

# Convert the extracted data to pandas DataFrames and adjust timestamps
def convert_df(data, timezone='Etc/GMT-12'):
    df = pd.DataFrame(data[1:], columns=data[0])
    df['timestamp'] = pd.to_datetime(df['time'], unit='ms').dt.tz_localize('UTC').dt.tz_convert(timezone)
    df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return df

pixel_values_df = convert_df(pixel_values)
cloud_values_df = convert_df(cloud_values)

# Function to extract sun angles from image metadata
def extract_sun_angles(image):
    # Extract the sun zenith and azimuth angles as ee.Number objects
    sun_zenith = ee.Number(image.get('MEAN_SOLAR_ZENITH_ANGLE'))
    sun_azimuth = ee.Number(image.get('MEAN_SOLAR_AZIMUTH_ANGLE'))
    # Return an ee.Feature with these properties for standardized handling
    return ee.Feature(None, {
        'system:time_start': image.get('system:time_start'), 
        'sun_zenith': sun_zenith, 
        'sun_azimuth': sun_azimuth
    })

# Apply the function to each image in the collection and fetch the data to the client side
sun_angles = images.map(extract_sun_angles).getInfo()

# Convert the sun angles information to a pandas DataFrame
sun_angles_df = pd.DataFrame([{
    'timestamp': pd.to_datetime(image['properties']['system:time_start'], unit='ms').tz_localize('UTC').tz_convert('Etc/GMT-12').strftime('%Y-%m-%d %H:%M:%S'), 
    'sun_zenith': image['properties']['sun_zenith'], 
    'sun_azimuth': image['properties']['sun_azimuth']
} for image in sun_angles['features']])

# Check and print unique coordinates returned by GEE
unique_coords = pixel_values_df[['longitude', 'latitude']].drop_duplicates()
print("Unique coordinates returned by GEE:")
print(unique_coords)

# Function to find the nearest points
def find_nearest_points(gee_points, poi_points):
    tree = cKDTree(poi_points)
    distances, indices = tree.query(gee_points)
    return indices

# Get unique GEE returned points and find the nearest POIs for each GEE returned point
gee_points = list(zip(pixel_values_df['longitude'].unique(), pixel_values_df['latitude'].unique()))
pixel_values_df['poi_index'] = find_nearest_points(list(zip(pixel_values_df['longitude'], pixel_values_df['latitude'])), points_of_interest)
cloud_values_df['poi_index'] = find_nearest_points(list(zip(cloud_values_df['longitude'], cloud_values_df['latitude'])), points_of_interest)

# Merge DataFrames based on the nearest POI and include sun angles
def merge_dataframes_by_poi(pixels_df, clouds_df, sun_df):
    merged_dfs = []
    for i in range(len(points_of_interest)):
        # Select rows corresponding to the current point of interest
        point_pixels = pixels_df[pixels_df['poi_index'] == i].drop(columns=['longitude', 'latitude', 'poi_index', 'time'])
        point_clouds = clouds_df[clouds_df['poi_index'] == i].drop(columns=['longitude', 'latitude', 'poi_index', 'time'])
        
        # Merge the pixel and cloud dataframes based on the timestamp
        point_df = pd.merge(point_pixels, point_clouds, on='timestamp', how='outer', suffixes=('_pixel', '_cloud'))
        point_df = point_df.add_prefix(f'point_{i+1}_').rename(columns={f'point_{i+1}_timestamp': 'timestamp'})
        
        # Drop redundant columns
        point_df = point_df.drop(columns=[f'point_{i+1}_id_cloud'])
        merged_dfs.append(point_df)
    
    # Concatenate all point DataFrames and remove duplicate columns
    merged_df = pd.concat(merged_dfs, axis=1)
    merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
    
    # Merge with sun angles DataFrame based on the timestamp
    merged_df = pd.merge(merged_df, sun_df, on='timestamp', how='left')
    
    return merged_df

# Merge dataframes and print the final result
final_df = merge_dataframes_by_poi(pixel_values_df, cloud_values_df, sun_angles_df)
print(final_df)



Unique coordinates returned by GEE:
       longitude   latitude
0     170.891412 -43.999258
800   170.891592 -43.999258
1600  170.891682 -43.999258
2400  170.891772 -43.999258
3200  170.891951 -43.999258
4000  170.891951 -43.999168
4800  170.891412 -43.999078
5600  170.891951 -43.999078
                            point_1_id_pixel  point_1_B2  point_1_B3  \
0                                        NaN         NaN         NaN   
1                                        NaN         NaN         NaN   
2                                        NaN         NaN         NaN   
3                                        NaN         NaN         NaN   
4                                        NaN         NaN         NaN   
...                                      ...         ...         ...   
1040  20240611T222549_20240611T222545_T59GMM      2002.0      2026.0   
1041  20240614T223709_20240614T223711_T59GMM       484.0       453.0   
1042  20240616T222551_20240616T222548_T59GMM      2908.0      28

In [11]:
final_df = final_df.dropna()
# Save the final DataFrame to a CSV file without including the index
final_df.to_csv('s2_new.csv', index=False)


In [8]:
final_df.set_index('timestamp', inplace=True)

In [9]:
final_df

,point_1_id_pixel,point_1_B2,point_1_B3,point_1_B4,point_1_B8,point_1_B8A,point_1_B11,point_1_B12,point_1_cs,point_1_cs_cdf,point_2_id_pixel,point_2_B2,point_2_B3,point_2_B4,point_2_B8,point_2_B8A,point_2_B11,point_2_B12,point_2_cs,point_2_cs_cdf,point_3_id_pixel,point_3_B2,point_3_B3,point_3_B4,point_3_B8,point_3_B8A,point_3_B11,point_3_B12,point_3_cs,point_3_cs_cdf,point_4_id_pixel,point_4_B2,point_4_B3,point_4_B4,point_4_B8,point_4_B8A,point_4_B11,point_4_B12,point_4_cs,point_4_cs_cdf,point_5_id_pixel,point_5_B2,point_5_B3,point_5_B4,point_5_B8,point_5_B8A,point_5_B11,point_5_B12,point_5_cs,point_5_cs_cdf,point_6_id_pixel,point_6_B2,point_6_B3,point_6_B4,point_6_B8,point_6_B8A,point_6_B11,point_6_B12,point_6_cs,point_6_cs_cdf,point_7_id_pixel,point_7_B2,point_7_B3,point_7_B4,point_7_B8,point_7_B8A,point_7_B11,point_7_B12,point_7_cs,point_7_cs_cdf,point_8_id_pixel,point_8_B2,point_8_B3,point_8_B4,point_8_B8,point_8_B8A,point_8_B11,point_8_B12,point_8_cs,point_8_cs_cdf,sun_zenith,sun_azimuth
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-12-14 10:38:15,20181213T223659_20181213T223656_T59GMM,2264.0,2252.0,2136.0,3318.0,3803.0,3274.0,2938.0,0.196078,0.227451,20181213T223659_20181213T223656_T59GMM,2416.0,2326.0,2118.0,3510.0,3496.0,3117.0,2582.0,0.203922,0.231373,20181213T223659_20181213T223656_T59GMM,2288.0,2442.0,2320.0,3536.0,3496.0,3117.0,2582.0,0.207843,0.235294,20181213T223659_20181213T223656_T59GMM,2300.0,2308.0,2208.0,3586.0,3668.0,3210.0,2712.0,0.215686,0.243137,20181213T223659_20181213T223656_T59GMM,2770.0,2668.0,2346.0,3992.0,3668.0,3210.0,2712.0,0.223529,0.247059,20181213T223659_20181213T223656_T59GMM,2820.0,2856.0,2520.0,4092.0,3668.0,3210.0,2712.0,0.211765,0.235294,20181213T223659_20181213T223656_T59GMM,3098.0,2924.0,2508.0,4080.0,3665.0,3308.0,2849.0,0.172549,0.203922,20181213T223659_20181213T223656_T59GMM,2620.0,2732.0,2564.0,3980.0,3837.0,3379.0,2901.0,0.203922,0.235294,31.254377,57.960822
2018-12-16 10:28:16,20181215T222531_20181215T222536_T59GMM,8392.0,7808.0,7360.0,7536.0,7384.0,5487.0,4050.0,0.003922,0.043137,20181215T222531_20181215T222536_T59GMM,8416.0,7816.0,7408.0,7572.0,7361.0,5451.0,4024.0,0.003922,0.043137,20181215T222531_20181215T222536_T59GMM,8336.0,7812.0,7472.0,7560.0,7361.0,5451.0,4024.0,0.003922,0.043137,20181215T222531_20181215T222536_T59GMM,8296.0,7808.0,7416.0,7520.0,7387.0,5445.0,4006.0,0.003922,0.043137,20181215T222531_20181215T222536_T59GMM,8336.0,7788.0,7388.0,7508.0,7387.0,5445.0,4006.0,0.003922,0.039216,20181215T222531_20181215T222536_T59GMM,8296.0,7728.0,7332.0,7500.0,7387.0,5445.0,4006.0,0.003922,0.039216,20181215T222531_20181215T222536_T59GMM,8480.0,7872.0,7448.0,7592.0,7505.0,5537.0,4104.0,0.003922,0.043137,20181215T222531_20181215T222536_T59GMM,8304.0,7736.0,7284.0,7480.0,7365.0,5410.0,3990.0,0.003922,0.039216,32.880865,61.583752
2018-12-19 10:38:13,20181218T223651_20181218T223734_T59GMM,6812.0,6896.0,6976.0,7540.0,7385.0,3879.0,2740.0,0.011765,0.058824,20181218T223651_20181218T223734_T59GMM,6812.0,6932.0,7020.0,7516.0,7430.0,3888.0,2768.0,0.011765,0.058824,20181218T223651_20181218T223734_T59GMM,6784.0,6904.0,7012.0,7508.0,7430.0,3888.0,2768.0,0.011765,0.058824,20181218T223651_20181218T223734_T59GMM,6808.0,6920.0,6988.0,7540.0,7387.0,3890.0,2775.0,0.011765,0.058824,20181218T223651_20181218T223734_T59GMM,6828.0,6916.0,7032.0,7520.0,7387.0,3890.0,2775.0,0.011765,0.054902,20181218T223651_20181218T223734_T59GMM,6828.0,6904.0,7032.0,7560.0,7387.0,3890.0,2775.0,0.011765,0.054902,20181218T223651_20181218T223734_T59GMM,6748.0,6904.0,7072.0,7508.0,7439.0,3880.0,2759.0,0.011765,0.058824,20181218T223651_20181218T223734_T59GMM,6828.0,6964.0,7032.0,7568.0,7437.0,3919.0,2792.0,0.011765,0.054902,31.473882,59.043502
2018-12-21 10:28:20,20181220T222539_20181220T222533_T59GMM,2152.0,2330.0,2066.0,3628.0,3513.0,2676.0,2317.0,0.047059,0.141176,20181220T222539_20181220T222533_T59GMM,2032.0,2258.0,2120.0,3542.0,3515.0,2757.0,2332.0,0.054902,0.149020,2018